In [ ]:
%%bash
wget https://raw.githubusercontent.com/TalSchuster/FeverSymmetric/refs/heads/master/symmetric_v0.2/fever_symmetric_dev.jsonl
wget https://raw.githubusercontent.com/TalSchuster/FeverSymmetric/refs/heads/master/symmetric_v0.2/fever_symmetric_test.jsonl

In [ ]:
import pandas as pd

dev_json='/kaggle/working/fever_symmetric_dev.jsonl'
test_json='/kaggle/working/fever_symmetric_test.jsonl'


df_dev = pd.read_json(dev_json, lines=True)
df_dev.head()

In [ ]:
df_test = pd.read_json(test_json, lines=True)

combine_df=pd.concat([df_dev,df_test], ignore_index=True).head(1200)
combine_df.shape

In [ ]:
# Shuffle with a fixed random seed
df = combine_df.sample(frac=1, random_state=42).reset_index(drop=True)
df.shape

In [ ]:
import os
import glob

folder_name='/kaggle/working/'
jsonl_files=glob.glob(os.path.join(folder_name,'*.jsonl'))

for f_p in jsonl_files:
    try:
        os.remove(f_p)
        print(f"Deleted: {f_p}")
    except Exception as e:
        print(f"Error deleting {f_p}: {e}")

# Building Reasoning Paths for FEVER

we will generate multiple resoning paths by using LLM and follow the SoT prompt strategy.

id	claim	evidence	sentence1	sentence2	gold_label	label	evid

In [6]:
from typing import Optional,List,Union
from pydantic import BaseModel

class Entity(BaseModel):
    id: int
    claim: str
    evidence: str
    sentence1: str
    sentence2: str
    gold_label: str
    label: str
    evid: List[List[Union[str, float]]]

class ReasoningPath(BaseModel):
    question: str
    evidence: str
    reason: str
    answer: Optional[str]=None
    ground_truth: str
    temperature: float
    tokens: int
    score: float

entities = [Entity.model_validate(row) for row in df.to_dict(orient='records')]

In [ ]:
import re
from sketch_of_thought import SoT
from typing import Tuple, Union
from doraemon import Doraemon

logger=Doraemon.get_logger(name=__name__, logfile="fever_dataset_builder.log")

def get_answer(raw_answer: str)-> str:
    answer=re.search(r'\\boxed{([^}]+)}', raw_answer)
    if answer:
        return answer.group(1)
    return None
    

def process_entity(args)-> Optional[ReasoningPath]:
    """
    """
    et,paradigm,temperature=args

    try:
        prompt=SoT.few_short_support(
            task_name='FEVER', 
            prompt=SoT.get_initialized_prompt(paradigm=paradigm), 
            few_short=SoT.get_few_short(task_name='FEVER'),
            question=f"Question:{et.claim}\nContext:{et.evidence}"
        )
        
        r_s, tokens=Doraemon.inference(
            logger=logger, 
            messages=prompt, 
            temperature=temperature
        )
        
        result=ReasoningPath(
            question=str(et.claim), 
            evidence=str(et.evidence), 
            reason=str(r_s), 
            answer=get_answer(r_s),
            ground_truth=str(et.gold_label),
            temperature=float(temperature), 
            tokens=int(tokens), 
            score=0.0)
        return result
    except Exception as e:
        logger.error(f"Error processing quetion {et.claim} at temperature {temperature} with exception {e}")
        return None


paradigm = SoT.classify_question(entities[0].claim)
logger.info(paradigm)
assert "conceptual_chaining"==str(paradigm)

In [8]:
temperatures = [i * 0.25 for i in range(9)]  # [0.0, 0.25, 0.5, ... ,2.0]

tasks=[]
for et in entities:
    for tp in temperatures:
        tasks.append((et,paradigm,tp))

In [ ]:
from tqdm import tqdm
from typing import List, Dict
import concurrent.futures

with concurrent.futures.ProcessPoolExecutor(max_workers=1) as executor:
    results=list(tqdm(executor.map(process_entity, tasks), total=len(tasks)))

# Convert Data into pkl File 

In [10]:
final_list:List[Dict] = [rp.model_dump() for rp in results if rp is not None]

final_df=pd.DataFrame(final_list)

mapping = {'true': 'SUPPORTS', 'false': 'REFUTES'}
final_df['answer'] = final_df['answer'].str.lower().map(mapping).fillna(final_df['answer'])

final_df.to_pickle('cots_df.pkl')

In [ ]:
import matplotlib.pyplot as plt

# Tokens distribution
plt.figure()
final_df['tokens'].hist()
plt.title("Tokens per Example")
plt.xlabel("tokens")
plt.ylabel("count")
plt.show()